In [1]:
from pymongo import MongoClient

In [2]:
client = MongoClient()

In [7]:
client.list_database_names()

['Listeria_monocytogenes_CType',
 'Neisseria_meningitidis',
 'admin',
 'config',
 'local']

In [6]:
client.drop_database('Vibrio_cholerae')

In [8]:
db = client['Vibrio_cholerae']

In [46]:
db.list_collection_names()

[]

In [45]:
db.drop_collection('cgMLST-profiles')

{'ns': 'Vibrio_cholerae.cgMLST-profiles', 'nIndexesWas': 1, 'ok': 1.0}

In [47]:
collection = db['cgMLST_profiles']

In [13]:
from pathlib import Path
import pandas as pd

In [50]:
dirpath = Path('/media/NGS/SRA_1/NCBI_Vibrio_cholerae_SRA/Profile')

In [51]:
profiles = {filepath.stem: pd.read_csv(filepath, sep='\t', index_col=0) for filepath in dirpath.iterdir()}

In [52]:
for accession, profile in profiles.items():
    profiles[accession] = profile.iloc[:, 0].dropna().to_dict()

In [53]:
documents = []
for accession, profile in profiles.items():
    document = dict()
    document['accession'] = accession
    document['profile'] = profile
    documents.append(document)

In [54]:
collection.insert_many(documents)

In [56]:
collection = db['sample_attribute']

In [58]:
docs = [doc for doc in db.cgMLST_profiles.find({}, {'_id': 0, 'profile': 0})]

In [63]:
collection.insert_many(docs)

In [84]:
amr = pd.read_csv('/media/NGS/SRA_1/NCBI_Vibrio_cholerae_SRA/AMR.csv')

In [85]:
amr = amr[amr['AMR genes'].notna()]

In [86]:
amr = dict(zip(amr['Key'], amr['AMR genes']))

In [87]:
amr = {accession: genes.split(', ') for accession, genes in amr.items()}

In [88]:
for accession, genes in amr.items():
    collection.update_one({'accession': accession}, {'$set': {'resistance_genes': genes}})

In [99]:
mlst = pd.read_csv('/media/NGS/Sequence/Vibrio_cholerae/MLST.csv', usecols=[0, 1])

In [102]:
for accession, st in zip(mlst['Key'], mlst['ST']):
    collection.update_one({'accession': accession}, {'$set': {'mlst': st}})

In [133]:
plasmid_type = pd.read_csv('/media/NGS/Sequence/Vibrio_cholerae/Plasmid.csv')

In [134]:
plasmid_type = plasmid_type[plasmid_type['Plasmid'].notna()]

In [136]:
for accession, inc_type in zip(plasmid_type['Key'], plasmid_type['Plasmid']):
    inc_type = inc_type.split(', ')
    collection.update_one({'accession': accession}, {'$set': {'inc_type': inc_type}})